### Nifty Major Sectors Analysis - For the last 5 years

Nifty's Sectoral Analysis, keeping the Nifty50 as the base index. Indices included are:

- Nifty Bank
- Nifty Financial Service
- Nifty Auto
- Nifty IT
- Nifty Metal
- Nifty FMCG
- Nifty Pharma
- Nifty HealthCare
- Nifty Consumer Durables

`NIFTY50` as the base index - to see and compare the performance

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Import Statements
import pandas as pd
from tqdm import tqdm
from datetime import date
import matplotlib.pyplot as plt

In [ ]:
## Function to resample the given dataframe on the timeframe provided
def resample_dataframe(df, timeframe):
    ohlc_aggregator = {
        "Open": "first",
        "High": "max",
        "Low": "min",
        "Close": "last"
    }
    df = df.resample(timeframe, on="Date").agg(ohlc_aggregator)
    df.dropna(inplace=True)
    return df

In [ ]:
## Main function to create a overall dataframe, with sectors as columns
def create_sectoral_dataframe(SECTORS_DATA_PATH, base_index, start_date, end_date, timeframe):
    ## Global dataframe
    overall_sector_df = pd.DataFrame()
    ## Open the Nifty DF & get the close
    nifty_df = pd.read_csv(f"{SECTORS_DATA_PATH}\\{base_index}_minute.csv", index_col=0)
    nifty_df["Date"] = pd.to_datetime(nifty_df["Date"])
    nifty_df = resample_dataframe(nifty_df, timeframe)
    nifty_df = nifty_df[start_date:end_date]
    ## Append the close to the overall df
    overall_sector_df = nifty_df[["Close"]]
    overall_sector_df.rename(columns={"Close":f"Close_{base_index}"}, inplace=True)
    ## Loop in to each sector
    for sector in tqdm(sectors_list):
        # print(f"********* SECTOR: {sector} ***********")
        sector_df = pd.read_csv(f"{SECTORS_DATA_PATH}\\{sector}_minute.csv", index_col=0)
        sector_df["Date"] = pd.to_datetime(sector_df["Date"])
        sector_df = resample_dataframe(sector_df, timeframe)
        sector_df = sector_df[start_date:end_date]
        overall_sector_df = pd.concat([overall_sector_df, sector_df[["Close"]]], axis=1)
        overall_sector_df.rename(columns={"Close":f"Close_{sector}"}, inplace=True)
    ## Apply fillna methods
    overall_sector_df.fillna(method="bfill", inplace=True)
    overall_sector_df.fillna(method="ffill", inplace=True)
    return overall_sector_df

In [ ]:
## Sectors list to include
sectors_list = ["NIFTY BANK", "NIFTY FMCG", "NIFTY FIN SERVICE", "NIFTY IT", "NIFTY METAL",
                "NIFTY AUTO", "NIFTY PHARMA", "NIFTY ENERGY", "NIFTY 100"]
base_index = "NIFTY 50"

In [ ]:
## Variables and file paths
SECTORS_DATA_PATH = "F:\\MyProjects\\StockStrategies\\SectorIndex_1min_Data"
start_date = date(2018,4,1)
end_date = date(2023,3,31)
timeframe = "D"
overall_sector_df = create_sectoral_dataframe(SECTORS_DATA_PATH, base_index, start_date, end_date, timeframe)

In [ ]:
overall_sector_df.head()

In [ ]:
### Overall Returns across each sector
sectoral_returns = {}
for column in overall_sector_df.columns:
    sector_name = column.split("_")[1]
    return_pct = round(((overall_sector_df.iloc[-1][column]-overall_sector_df.iloc[0][column])/overall_sector_df.iloc[0][column]*100), 3)
    sectoral_returns[sector_name] = return_pct
sectoral_returns

### Visualisation - Graphs

In [ ]:
## Matplotlib to create a graph - for the first 5 indices (more common)
plt.figure(figsize=(12, 6))
## Plot the line graph
plt.plot(overall_sector_df.index, overall_sector_df[f"Close_{base_index}"], color="black", linewidth=3, label=base_index)
for column in overall_sector_df.columns[1:5]:
    plt.plot(overall_sector_df.index, overall_sector_df[column], label=column.split("_")[1])
## Adding labels and titles
plt.xlabel("Date")
plt.ylabel("Daily Closing Price")
plt.title("Sectoral Performance of Different Indices")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
## Matplotlib to create a graph - for the other 5 indices (less common)
plt.figure(figsize=(12, 6))
## Plot the line graph
plt.plot(overall_sector_df.index, overall_sector_df[f"Close_{base_index}"], color="black", linewidth=3, label=base_index)
for column in overall_sector_df.columns[5:]:
    plt.plot(overall_sector_df.index, overall_sector_df[column], label=column.split("_")[1])
## Adding labels and titles
plt.xlabel("Date")
plt.ylabel("Daily Closing Price")
plt.title("Sectoral Performance of Different Indices")
plt.legend(loc="best")
plt.tight_layout()
plt.show()

In [ ]:
## Matplotlib to create a bar graph - of the returns over the last years
plt.figure(figsize=(10, 5))
## Plot the bar graph
plt.bar(sectoral_returns.keys(), sectoral_returns.values(), color="green")
for category, value in zip(sectoral_returns.keys(), sectoral_returns.values()):
    plt.text(category, value, str(value), ha='center', va='bottom')
## Adding labels and titles
plt.xlabel("Sectors")
plt.ylabel("Returns (In pct)")
plt.title("Sectoral Returns of Different Indices")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()